# Default building control using an empty action space

It is possible to run a simulation using the default building control performed by EnergyPlus (as specified in the building model file).

For instance, from the container's workspace, run the following command:


```bash
$ energyplus -w sinergym/data/weather/USA_PA_Pittsburgh-Allegheny.County.AP.725205_TMY3.epw sinergym/data/buildings/5ZoneAutoDXVAV.epJSON
```

However, doing this without our framework has some **drawbacks**:

- You will only have the default EnergyPlus output and will not have access to **additional output information**, such as data provided by the logger wrapper, which tracks all the environment interactions. 

- Moreover, building models have a default ``Site:Location``and ``SizingPeriod:DesignDay``, which *Sinergym* automatically adjusts based on the specified weather, so you would need to manually modify these settings before launching the simulation.

- Lastly, you would also need to manually adjust the *RunPeriod* in the building file before starting the simulation.

Hence, to avoid manual configurations, we recommend setting up an **empty action interface** in a *Sinergym* environment. For instance:

In [1]:
import gymnasium as gym
import numpy as np

import sinergym
from sinergym.utils.wrappers import LoggerWrapper

env = gym.make(
    'Eplus-office-hot-continuous-v1',
    actuators={},
    action_space=gym.spaces.Box(
        low=0,
        high=0,
        shape=(0,)))
env = LoggerWrapper(env)

for i in range(1):
    obs, info = env.reset()
    rewards = []
    truncated = terminated = False
    current_month = 0
    while not (terminated or truncated):
        a = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(a)
        rewards.append(reward)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', sum(rewards), info)
    print(
        'Episode ',
        i,
        'Mean reward: ',
        np.mean(rewards),
        'Cumulative reward: ',
        sum(rewards))
env.close()

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment.
[ENVIRONMENT] (INFO) : Name: office-hot-continuous-v1
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created.
[MODELING] (INFO) : Working directory: /workspaces/sinergym/examples/Eplus-env-office-hot-continuous-v1-res1
[MODELING] (INFO) : Model Config is correct.
[MODELING] (INFO) : Update building model Output:Variable with variable names.
[MODELING] (INFO) : Update building model Output:Meter with meter names.
[MODELING] (INFO) : Runperiod established.
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[REWARD] (INFO) : Reward function initialized.
[ENVIRONMENT] (INFO) : Environment created successfully.
[WRAPPER LoggerWrapper] (INFO

In this example, a default environment is created, but the space and definition of the default action **are replaced with an empty one**. *Sinergym* handles the necessary background changes. Then, the implemented random agent sends **empty actions (`[]`)** to the environment.

When setting an empty action space, *Sinergym* retains the default actuators that are defined in the building model. Their complexity and implementation will depend on the building definition in the *epJSON* file.

The benefits of this approach include the ability to **mix and match weathers and buildings** as desired, with *Sinergym* automatically making all the necessary configuration.

You can simulate as many years as you want in a single experiment, using the pre-defined loggers.

This method also provides more flexibility when choosing which observation variables are going to be used. 